In [ ]:

# put the ticker names back into the test dataset
   
test['Ticker'] = test['Ticker'].astype('category')
# make y_pred and y_pred_proba a column in the test dataset
test['Prediction'] = y_pred
test['Prediction Probability (0)'] = y_pred_proba[:,0]
test['Prediction Probability (1)'] = y_pred_proba[:,1]
# only keep columns for ticker, target, and prediction
test = test[['Ticker', 'Target', 'Prediction', 'Prediction Probability (1)', 'Prediction Probability (0)', 'Return']]
# make the ticker column a name again
# make a new dataframe where return is the index and each column is a ticker
return_df = test.pivot(columns='Ticker', values='Return')
return_df

# for each ticker only keep the first row
test = test.groupby('Ticker').head(1)
test

In [ ]:
# make a new dataframe where return is the index and each column is a ticker
# only keep the last 10 rows
return_deviation_precovid = return_df.head(62)

return_deviation_precovid

In [ ]:
# make covariance matrix of returns of the stocks
cov_matrix_precovid =  return_deviation_precovid.cov()
cov_matrix_precovid

In [ ]:
n = len(test["Ticker"])  # number of stocks
w = cp.Variable(n)  # weights for each stock in the portfolio

# Objective is to minimize portfolio variance
risk = cp.quad_form(w, cov_matrix_precovid)
objective = cp.Minimize(risk)

# Constraints:
# - weights must sum to 1
# - only consider stocks where the SVM prediction is 1 (price will increase)
# - expected portfolio return must be at least a certain value (e.g., 0.01)
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    w[test["Prediction"] == 0] == 0,
    w.T @ test["Prediction Probability (1)"] >= 0.01
]

# Solve the optimization problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Get the optimal weights
optimal_weights = w.value

In [ ]:
# put the optimal weights into a dataframe with the tickers
optimal_weights_df = pd.DataFrame(optimal_weights, index=test["Ticker"])
optimal_weights_df
# add the ticer names to the dataframe
optimal_weights_df = optimal_weights_df.join(df)
# remove the index
optimal_weights_df = optimal_weights_df.reset_index(drop=True)
# make the ticker the index
optimal_weights_df = optimal_weights_df.set_index('Ticker')
# make the column name weights
optimal_weights_df.columns = ['Weights']
# sort the dataframe by the optimal weights
optimal_weights_df = optimal_weights_df.sort_values(by=['Weights'], ascending=False)
optimal_weights_df

In [ ]:
# only keep the top 7 stocks and their weights
optimal_weights_df = optimal_weights_df.head(7)
optimal_weights_df

In [ ]:
sum(optimal_weights_df['Weights'])

In [ ]:
# S&P 500 index return in the period:
start_p = 2960.60
end_p = 3087.41
# calculate the return in percent
sp_return = (end_p - start_p) / start_p * 100
sp_return

In [ ]:
# Dictionary with buy price for each stock
buy_price = {'AMZN': 91.10, 'WFC': 47.31, 'MO': 44.55, 'BA': 360.14,'ADM': 38.50, 'F': 9.25, 'TSLA': 14.83}
# Dictionary with sell price for each stock
sell_price = {'AMZN': 88.00, 'WFC': 53.01, 'MO': 50.46, 'BA': 352.09,'ADM': 42.89, 'F': 8.95, 'TSLA': 22.17}

# Calculate the return for each stock of the dictioniary in percent
return_dict = {}
for key in buy_price.keys():
    return_dict[key] = (sell_price[key] - buy_price[key]) / buy_price[key] * 100
return_dict

In [ ]:
# Calculate the return of the portfolio
portfolio_return = 0
for key in optimal_weights_df.index:
    portfolio_return += return_dict[key] * optimal_weights_df.loc[key]['Weights']

print(f'The return of the portfolio is: {round(portfolio_return,2)}%\nThis is compared to a return of {round(sp_return,2)}% for the S&P 500 index in the same period.')

In [ ]:
# We will find 3 month return for the SP500 for each day in the testing period:

In [33]:
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf

# This line is needed to fix a problem with pandas_datareader
yf.pdr_override()

start_date = '2019-01-01'
end_date = '2020-04-03'
# 
# Get data from Yahoo Finance
sp500 = pdr.get_data_yahoo('^GSPC', start=start_date, end=end_date)
# we are only interested in the open price
sp500 = sp500['Open']
# Transform it into a dataframe
sp500_returns = pd.DataFrame(sp500)
# Make 'S&P 500 Index Price' the column header
sp500_returns = sp500_returns.rename(columns={'Open': 'S&P 500 Index Price'})
# Calculate the 3 month return for each day
sp500_returns['3 Month Return (%)'] = sp500_returns['S&P 500 Index Price'].pct_change(periods=62).shift(-62)
# calculating all 3-month return values by 100 to obtain percent
sp500_returns['3 Month Return (%)'] = sp500_returns['3 Month Return (%)'] * 100
sp500_returns.round(2)
# remove the S&P 500 Index Price column
sp500_returns = sp500_returns.drop(columns=['S&P 500 Index Price'])
# removing all rows with NaN values (The last 62 days, since they don't have data for 3 months into the future)
sp500_returns = sp500_returns.dropna()


[*********************100%***********************]  1 of 1 completed


In [ ]:
#non-working version:
import cvxpy as cp
import pandas as pd

def calculate_optimal_weights(test, date, y_pred, y_pred_proba,ticker_symbols):
    # only keep the rows of a specific date
    # make a date be of type datetime
    date = pd.to_datetime(date)
    
    df_date = test.copy()
    print(df_date['Ticker'].unique().shape)


    # Add the predictions and prediction probabilities to the dataframe
    df_date['Prediction'] = y_pred
    df_date['Prediction Probability (0)'] = y_pred_proba[:,0]
    df_date['Prediction Probability (1)'] = y_pred_proba[:,1]
    
   # only keep columns for ticker, target, and prediction
    df_date = df_date[['Ticker', 'Target', 'Prediction', 'Prediction Probability (1)', 'Prediction Probability (0)', 'Return']]
    print(df_date['Ticker'].unique().shape)
    # Create a dataframe with return as the index and each column as a ticker
    return_df = df_date.pivot(columns='Ticker', values='Return')
    print(return_df)
    return_deviation = return_df.head(62)
    print(return_deviation)
    # Filter the dataframe for the given date
    #df_date = df_date[df_date.index == date]
    df_date = df_date[pd.DatetimeIndex(df_date.index).date == date]
    print(return_deviation.shape, df_date.shape, return_deviation)


    

    # Calculate the covariance matrix
    cov_matrix = return_deviation.cov()
    # Force the covariance matrix to be symmetric
    cov_matrix = (cov_matrix + cov_matrix.T) / 2
    cov_matrix = cov_matrix.round(10)

    print(cov_matrix, cov_matrix.shape)
    # Number of stocks
    #n = len(df_date["Ticker"])
    n = len(ticker_symbols)

    # Weights for each stock in the portfolio
    w = cp.Variable(n)
    #print(w.shape, cov_matrix.shape,n)
    # Objective is to minimize portfolio variance
    risk = cp.quad_form(w, cov_matrix)
    objective = cp.Minimize(risk)

    # Constraints:
    # - weights must sum to 1
    # - only consider stocks where the SVM prediction is 1 (price will increase)
    # - expected portfolio return must be at least a certain value (e.g., 0.01)
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
        w[df_date["Prediction"] == 0] == 0,
        w.T @ df_date["Prediction Probability (1)"] >= 0.05
    ]
    

    # Solve the optimization problem
    problem = cp.Problem(objective, constraints)
    problem.solve()

    # Get the optimal weights
    optimal_weights = w.value

    # Create a dataframe with the optimal weights
    optimal_weights_df = pd.DataFrame(optimal_weights, index=df_date["Ticker"])
    # add the ticker names to the dataframe
    optimal_weights_df = optimal_weights_df.join(ticker_name_df)
    # remove the index
    optimal_weights_df = optimal_weights_df.reset_index(drop=True)
    # make the ticker the index
    optimal_weights_df = optimal_weights_df.set_index('Ticker')
    # make the column name weights
    optimal_weights_df.columns = ['Weights']
   


    # Sort the dataframe by the optimal weights
    optimal_weights_df = optimal_weights_df.sort_values(by=['Weights'], ascending=False)

    # Only keep the top 10 stocks and their weights
    optimal_weights_df = optimal_weights_df.head(10)

    return round(optimal_weights_df,3)
